In [20]:
import os
%load_ext dotenv
%dotenv
key = os.environ.get('BING_KEY')
from fastbook import *

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [31]:
results = search_images_bing(key, 'timothee chalamet')
ims = results.attrgot('contentUrl')
len(ims)

150

In [38]:
friends = ['ben', 'elizabeth', 'andy', 'sam', 'stef', 'haylee']
doppelgangers = ['timothee chalamet', 'florence pugh', 'ed helms', 'ryan eggold', 'elizabeth olsen', 'zooey deschanel']
path = Path('data')

In [40]:
if not path.exists():
    path.mkdir()
    for friend, doppelganger in zip(friends, doppelgangers):
        dest = (path/friend)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, doppelganger)
        download_images(dest, urls=results.attrgot('contentUrl'))